In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('googleplaystore.csv')

# loading 3 columns of the dataset

In [3]:
df.head(3)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up


# Checking for the null values

In [4]:
df.isnull()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10836,False,False,False,False,False,False,False,False,False,False,False,False,False
10837,False,False,False,False,False,False,False,False,False,False,False,False,False
10838,False,False,True,False,False,False,False,False,False,False,False,False,False
10839,False,False,False,False,False,False,False,False,False,False,False,False,False


# Check for null values in each column 

In [5]:
df.isnull().sum()

App                  0
Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       1
Genres               0
Last Updated         0
Current Ver          8
Android Ver          3
dtype: int64

In [6]:
#Since there is only 1 null value in Content rating, dropping it is a suitable process. It is because the data is large.

In [7]:
df.dropna(subset=['Content Rating'], inplace=True) # inplace = True makes changes in the original dataset


In [8]:
# We again check for the null values

In [9]:
df.isnull().sum()

App                  0
Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       0
Genres               0
Last Updated         0
Current Ver          8
Android Ver          2
dtype: int64

In [10]:
# Now that we have cleaned the column named content rating , we can also see that the column 'Type' , 'Android Version'and 'Current ver'
#also have minimum number 
# of missing values i.e 1 & 2 , so they can also be dropped as before.

In [11]:
df.dropna(subset=['Type'], inplace=True) ## we use inplace = true to make changes in the original dataset and not create a copy
df.dropna(subset=['Android Ver'], inplace=True)
df.dropna(subset = ['Current Ver'],inplace = True)

In [12]:
# Check again if the dataset is clean 

In [13]:
df.isnull().sum()

App                  0
Category             0
Rating            1469
Reviews              0
Size                 0
Installs             0
Type                 0
Price                0
Content Rating       0
Genres               0
Last Updated         0
Current Ver          0
Android Ver          0
dtype: int64

In [14]:
#We can still see null values in the column 'Ratings' which is 1473 and cannot be removed as earlier.Since ratings maybe skewed
# A better approach is to take median value and fill it in the null values for eg (rating : 1,4,4,4,4,5 ) then median = 4 and mean = 22/6 = 3.67
# So, we use median and use it in place of null values

In [15]:
# We do not take a single median and fill in the null values , but , instead we group the categories , find median and fill accordingly

In [16]:
df['Rating'] = df.groupby('Category')['Rating'].transform(
    lambda x: x.fillna(x.median())
) 
# Takes category eg : arts , finds the median of the rating in that category , and fills
#in the null values for the same category with its median. I used lambda function for simplicity here.

In [17]:
df.isnull().sum()

App               0
Category          0
Rating            0
Reviews           0
Size              0
Installs          0
Type              0
Price             0
Content Rating    0
Genres            0
Last Updated      0
Current Ver       0
Android Ver       0
dtype: int64

In [18]:
# Now our dataset is free from null values , and we move to the other step which is finding outliers in our data

In [19]:
# I started by checking for outliers in the 'price' column using the IQR method 

In [20]:
#Q1 = df['Price'].quantile(0.25)
#Q3 = df['Price'].quantile(0.75)
#IQR = Q3 - Q1

#lower_bound = Q1 - 1.5 * IQR
#upper_bound = Q3 + 1.5 * IQR

#outliers = df[(df['Price'] < lower_bound) | (df['Price'] > upper_bound)]
#print(outliers)


In [21]:
# I got to know that my price column is in string format and not integer , so i need to clean it 

In [22]:
#df['Price'] = df['Price'].astype(float)

In [23]:
#  $ symbol or commas are  removed 
df['Price'] = df['Price'].str.replace('$','', regex=False).str.replace(',','', regex=False)

# Converting to float value
df['Price'] = df['Price'].astype(float)


In [24]:
# Now check for outliers again

In [25]:
Q1 = df['Price'].quantile(0.25)
Q3 = df['Price'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['Price'] < lower_bound) | (df['Price'] > upper_bound)]

In [26]:
outliers.shape[0]


797

In [27]:
# Since the number of outliers is high, deleting it not an option.
# So i used the log transformation method , since log(x) is not defined for x<=0 , we need to ensure that our column has no such values


In [28]:
num = df[df['Price'] <= 0].shape[0]

In [29]:
num

10032

In [30]:
# So there are many values less than or equal to 0 , so prices cannot be negative so we makeshift it by adding 1 

In [31]:
import numpy as np
df['Price_log'] = np.log(df['Price'] + 1)

In [32]:
# Check for remaining outliers now 

In [33]:
Q1 = df['Price_log'].quantile(0.25)
Q3 = df['Price_log'].quantile(0.75)
IQR = Q3 - Q1


lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR


num_outliers = ((df['Price_log'] < lower_bound) | (df['Price_log'] > upper_bound)).sum()
num_outliers

np.int64(797)

In [34]:
# It is important to not delte these values now .Cause i can retain all the apps , both expensive and cheaper ones

# Now we are doing other simpler cleaning methods , now that the outliers and null values have been well handled    

In [35]:
df['Category'].value_counts()


Category
FAMILY                 1968
GAME                   1144
TOOLS                   841
MEDICAL                 463
BUSINESS                460
PRODUCTIVITY            424
PERSONALIZATION         390
COMMUNICATION           387
SPORTS                  384
LIFESTYLE               382
FINANCE                 366
HEALTH_AND_FITNESS      341
PHOTOGRAPHY             335
SOCIAL                  295
NEWS_AND_MAGAZINES      283
SHOPPING                260
TRAVEL_AND_LOCAL        258
DATING                  234
BOOKS_AND_REFERENCE     230
VIDEO_PLAYERS           175
EDUCATION               156
ENTERTAINMENT           149
MAPS_AND_NAVIGATION     137
FOOD_AND_DRINK          127
HOUSE_AND_HOME           88
AUTO_AND_VEHICLES        85
LIBRARIES_AND_DEMO       84
WEATHER                  82
EVENTS                   64
ART_AND_DESIGN           64
PARENTING                60
COMICS                   60
BEAUTY                   53
Name: count, dtype: int64

In [36]:
# There seems to be no  error in the spelling and spacing for categories so it may not be a topic of issue

In [37]:
df['Genres'].value_counts()


Genres
Tools                                840
Entertainment                        622
Education                            548
Medical                              463
Business                             460
                                    ... 
Parenting;Brain Games                  1
Travel & Local;Action & Adventure      1
Lifestyle;Pretend Play                 1
Tools;Education                        1
Strategy;Creativity                    1
Name: count, Length: 119, dtype: int64

In [38]:
# Deleting duplicate values for the column app

In [39]:
df['App'].duplicated().sum()


np.int64(1181)

In [40]:
# Since the versions are different for each duplicate app , we let it be

# Changing the datetime format to a standard format

In [41]:
df['Last Updated'] = pd.to_datetime(df['Last Updated'])

In [42]:
df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Price_log
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0.0,Everyone,Art & Design,2018-01-07,1.0.0,4.0.3 and up,0.0
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0.0,Everyone,Art & Design;Pretend Play,2018-01-15,2.0.0,4.0.3 and up,0.0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0.0,Everyone,Art & Design,2018-08-01,1.2.4,4.0.3 and up,0.0
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0.0,Teen,Art & Design,2018-06-08,Varies with device,4.2 and up,0.0
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0.0,Everyone,Art & Design;Creativity,2018-06-20,1.1,4.4 and up,0.0


In [43]:
df['Type'].value_counts()


Type
Free    10032
Paid      797
Name: count, dtype: int64

# Fixing Datatype now

In [44]:
df['Reviews'] = pd.to_numeric(df['Reviews'], errors='coerce')

In [45]:
df.dtypes

App                       object
Category                  object
Rating                   float64
Reviews                    int64
Size                      object
Installs                  object
Type                      object
Price                    float64
Content Rating            object
Genres                    object
Last Updated      datetime64[ns]
Current Ver               object
Android Ver               object
Price_log                float64
dtype: object

In [46]:
# Installs should be a numeric datatype and not an object

In [47]:
df['Installs'] = df['Installs'].str.replace('+','', regex=False)\
                               .str.replace(',','', regex=False) # we Use regex=False when:Replacing simple characters like $, ,, +
df['Installs'] = pd.to_numeric(df['Installs'], errors='coerce')


In [48]:
df.dtypes

App                       object
Category                  object
Rating                   float64
Reviews                    int64
Size                      object
Installs                   int64
Type                      object
Price                    float64
Content Rating            object
Genres                    object
Last Updated      datetime64[ns]
Current Ver               object
Android Ver               object
Price_log                float64
dtype: object

In [49]:
# Now every of the columns have the dtype they are supposed to have. 

# Final check if cleaning was sucessful

df.info()
df.isnull().sum()
df.describe()


# Using groupby for finding the average rating by category 

In [56]:
avg_rating = df.groupby('Category')['Rating'].mean().reset_index()
avg_rating

,Category,Rating
0,ART_AND_DESIGN,4.378125
1,AUTO_AND_VEHICLES,4.205882
2,BEAUTY,4.283019
3,BOOKS_AND_REFERENCE,4.380870
4,BUSINESS,4.182391
5,COMICS,4.163333
6,COMMUNICATION,4.180103
7,DATING,3.992308
8,EDUCATION,4.389103
9,ENTERTAINMENT,4.126174


# Total Reviews by content rating

In [54]:
total_reviews = df.groupby('Content Rating')['Reviews'].sum().reset_index()
total_reviews

,Content Rating,Reviews
0,Adults only 18+,81348
1,Everyone,2801822515
2,Everyone 10+,683997228
3,Mature 17+,197166533
4,Teen,1131523721
5,Unrated,1187


# Pivot table for  Total Installs by Category and Content Rating

In [58]:
pivot_installs = df.pivot_table(index='Category', columns='Content Rating', values='Installs', aggfunc='sum')
pivot_installs

Content Rating,Adults only 18+,Everyone,Everyone 10+,Mature 17+,Teen,Unrated
Category,,,,,,
ART_AND_DESIGN,NaN,6.382310e+07,5.000000e+05,NaN,6.001000e+07,NaN
AUTO_AND_VEHICLES,NaN,4.213021e+07,1.000000e+06,NaN,1.000000e+07,NaN
BEAUTY,NaN,2.671705e+07,1.100000e+05,1.200000e+05,2.500000e+05,NaN
BOOKS_AND_REFERENCE,NaN,2.412426e+08,2.111000e+07,5.501000e+06,1.653615e+09,NaN
BUSINESS,NaN,1.001812e+09,1.000000e+02,5.000000e+00,1.029300e+05,NaN
COMICS,1000000.0,1.247610e+07,5.100500e+05,2.310000e+06,3.979000e+07,NaN
COMMUNICATION,NaN,3.013255e+10,NaN,3.801100e+08,2.134617e+09,NaN
DATING,NaN,2.027896e+06,NaN,2.602520e+08,2.030910e+06,NaN
EDUCATION,NaN,8.064020e+08,6.200000e+07,2.050000e+06,1.000000e+06,NaN


# Let us now save the dataset which is cleaned

In [51]:
df.to_csv('google_play_cleaned.csv', index=False)

# CONCLUSION

In [52]:
# The Google Play Store dataset was carefully cleaned to
#make it reliable for analysis.
#Missing values were handled appropriately, incorrect data types were fixed,
#and important columns such as ratings, reviews, installs, prices, and dates were
#converted into usable formats. Outliers in price were identified, and a 
#log transformation was applied to reduce skewness. After cleaning, the dataset
#contained no missing values and had consistent data types,
#making it ready for exploratory analysis .